<a href="https://colab.research.google.com/github/rtakimoto/flight_delay/blob/main/tree/main/api/MachineLearning/notebooks/SmallFlightDelay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Geracao de Modelo para analise de atraso de voo
Instalando as bibliotecas

Bibliotecas necessárias

In [ ]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import os
import kagglehub
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

: 

#Carregamento do dataset (carga de um dataset pequeno para o MVP da PUC)



In [ ]:
# Informa a URL de importação do dataset
DatasetPath = kagglehub.dataset_download("eminhashimi/flight-delay-dataset")
print("Path to dataset files:", DatasetPath)

In [ ]:
FlightDataset = pd.read_csv(DatasetPath+"/flights_test.csv", delimiter=',')

# displaying the contents of the Test XLSX file
FlightDataset.head()

##Pre Processamento do dataset carregado

In [ ]:
FlightDataset.info()

# Remove rows with null values
FlightDatasetCleaned = FlightDataset.dropna()

In [ ]:
print('Shape of the resulting Flight Dataset table: ', FlightDataset.shape)
print('Shape of the resulting Flight Dataset table: ', FlightDatasetCleaned.shape)

In [ ]:
FlightDatasetCleaned['DELAY_DETECTED'] = 0
FlightDatasetCleaned.loc[(FlightDatasetCleaned['ARRIVAL_TIME']-FlightDatasetCleaned['SCHEDULED_ARRIVAL']) >0, 'DELAY_DETECTED'] = 1
FlightDatasetCleaned.head()

##Diminuição do número de registros para o MVP da PUC

In [ ]:
FlightDatasetCleaned = FlightDatasetCleaned.sample(frac=0.02)
print('Shape of the resulting Flight Dataset table: ', FlightDatasetCleaned.shape)
FlightDatasetCleaned.head()

In [ ]:
airline_names = FlightDatasetCleaned['AIRLINE'].unique()
tail_names = FlightDatasetCleaned['TAIL_NUMBER'].unique()
origin_names = FlightDatasetCleaned['ORIGIN_AIRPORT'].unique()
destination_names = FlightDatasetCleaned['DESTINATION_AIRPORT'].unique()

str_cols = FlightDatasetCleaned.columns[FlightDatasetCleaned.columns.str.contains('(?:AIRLINE|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT)')]
clfs = {c:LabelEncoder() for c in str_cols}

for col, clf in clfs.items():
  FlightDatasetCleaned[col] = clfs[col].fit_transform(FlightDatasetCleaned[col])

airline_values = FlightDatasetCleaned['AIRLINE'].unique()
tail_values = FlightDatasetCleaned['TAIL_NUMBER'].unique()
origin_values = FlightDatasetCleaned['ORIGIN_AIRPORT'].unique()
destination_values = FlightDatasetCleaned['DESTINATION_AIRPORT'].unique()

FlightDatasetCleaned.head()

In [ ]:
map_airline = np.array([airline_names, airline_values])
column_names = ['AIRLINE', 'VALUES']
airline_dataframe = pd.DataFrame(map_airline.transpose(), columns=column_names)
print(airline_dataframe)

map_tail = np.array([tail_names, tail_values])
column_names = ['TAIL_NUMBER', 'VALUES']
tail_dataframe = pd.DataFrame(map_tail.transpose(), columns=column_names)
print(tail_dataframe)

map_origin = np.array([origin_names, origin_values])
column_names = ['ORIGIN_AIRPORT', 'VALUES']
origin_dataframe = pd.DataFrame(map_origin.transpose(), columns=column_names)
print(origin_dataframe)

map_destination = np.array([destination_names, destination_values])
column_names = ['DESTINATION_AIRPORT', 'VALUES']
destination_dataframe = pd.DataFrame(map_destination.transpose(), columns=column_names)
print(destination_dataframe)

# Define o path
directory = 'data'

#Salvando os mapeamentos
if not os.path.exists(directory):
  os.makedirs(directory)

airline_dataframe.to_csv(os.path.join(directory, 'airline.csv'), index=False)
tail_dataframe.to_csv(os.path.join(directory, 'tail.csv'), index=False)
origin_dataframe.to_csv(os.path.join(directory, 'origin.csv'), index=False)
destination_dataframe.to_csv(os.path.join(directory, 'destination.csv'), index=False)


#print(clfs['AIRLINE'].inverse_transform(FlightDatasetCleaned['AIRLINE']))


# Separação em conjunto de treino e conjunto de teste com holdout

In [ ]:
test_size = 0.20 # tamanho do conjunto de teste
seed = 7 # semente aleatória

# Separação em conjuntos de treino e teste
array = FlightDatasetCleaned.values
X = FlightDatasetCleaned[['DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DEPARTURE_DELAY', 'SCHEDULED_ARRIVAL']].values
y = FlightDatasetCleaned['DELAY_DETECTED'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=True, random_state=seed, stratify=y) # holdout com estratificação

# Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 10
kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed) # validação cruzada com estratificação

#Modelagem e Inferência
Criação e avaliação de modelos

In [ ]:
np.random.seed(7) # definindo uma semente global

# Lista que armazenará os modelos
models = []

# Criando os modelos e adicionando-os na lista de modelos
models.append(('LR', LogisticRegression(max_iter=200)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# Definindo os parâmetros do classificador base para o BaggingClassifier
base = DecisionTreeClassifier()
num_trees = 100
max_features = 3

# Criando os modelos para o VotingClassifier
bases = []
model1 = LogisticRegression(max_iter=200)
bases.append(('logistic', model1))
model2 = DecisionTreeClassifier()
bases.append(('cart', model2))
model3 = SVC()
bases.append(('svm', model3))

# Criando os ensembles e adicionando-os na lista de modelos
models.append(('Bagging', BaggingClassifier(estimator=base, n_estimators=num_trees)))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('Ada', AdaBoostClassifier(n_estimators=num_trees)))
models.append(('GB', GradientBoostingClassifier(n_estimators=num_trees)))
models.append(('Voting', VotingClassifier(bases)))

# Listas para armazenar os resultados
results = []
names = []

# Avaliação dos modelos (treinamento)
for name, model in models:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(15,10))
fig.suptitle('Comparação dos Modelos')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

# Criação e avaliação de modelos: dados padronizados e normalizados

Trabalhando com dados pontencialmente desbalanceados ou sensíveis a escala

StandardScaler (padronização do conjunto de dados) e MinMaxScaler (normalização do conjunto de dados) são duas técnicas de normalização/escala usadas em machine learning para pré-processamento de dados e são úteis para preparar dados para algoritmos de aprendizado de máquina que são sensíveis à escala dos dados.

## StandardScaler
StandardScaler padroniza os dados, ou seja, remove a média e escala os dados para que tenham uma variância unitária. Ele transforma os dados para que a média de cada feature seja 0 e a variância seja 1.

Fórmula: $z_i=\frac{x_i-\mu}{\sigma}$


onde:
- $x_i$ é o valor original do $i$-ésimo termo da feature.
- $\mu$ é a média dos valores da feature.
- $\sigma$ é o desvio padrão dos valores da feature.
𝑥
x é o valor original da feature.
𝜇
μ é a média dos valores da feature.
𝜎
σ é o desvio padrão dos valores da feature.


## MinMaxScaler
MinMaxScaler escala e transforma os dados para um intervalo específico, geralmente entre 0 e 1. Ele transforma os dados para que o menor valor de uma feature seja 0 e o maior valor seja 1.

Fórmula: $z_i=\frac{x_i-min(x)}{max(x)-min(x)}$

onde:
- $x_i$ é o valor original do $i$-ésimo termo da feature.
- $min(x)$ é o menor valor da feature.
- $max(x)$ é o maior valor da feature.

Nós vamos aplicar essas técnicas para os dados do dataset de diabetes através da construção de pipelines. Pipelines são uma maneira de simplificar o processo de construção de modelos, permitindo que você execute várias etapas de pré-processamento e modelagem em sequência.

In [ ]:
np.random.seed(7) # definindo uma semente global para este bloco

# Listas para armazenar os armazenar os pipelines e os resultados para todas as visões do dataset
pipelines = []
results = []
names = []


# Criando os elementos do pipeline

# Algoritmos que serão utilizados
reg_log = ('LR', LogisticRegression(max_iter=200))
knn = ('KNN', KNeighborsClassifier())
cart = ('CART', DecisionTreeClassifier())
naive_bayes = ('NB', GaussianNB())
svm = ('SVM', SVC())
bagging = ('Bag', BaggingClassifier(estimator=base, n_estimators=num_trees))
random_forest = ('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features))
extra_trees = ('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features))
adaboost = ('Ada', AdaBoostClassifier(n_estimators=num_trees))
gradient_boosting = ('GB', GradientBoostingClassifier(n_estimators=num_trees))
voting = ('Voting', VotingClassifier(bases))

# Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())


# Montando os pipelines
# A ordem de execução é da esquerda para a direita.

# Dataset original
pipelines.append(('LR-orig', Pipeline([reg_log])))
pipelines.append(('KNN-orig', Pipeline([knn])))
pipelines.append(('CART-orig', Pipeline([cart])))
pipelines.append(('NB-orig', Pipeline([naive_bayes])))
pipelines.append(('SVM-orig', Pipeline([svm])))
pipelines.append(('Bag-orig', Pipeline([bagging])))
pipelines.append(('RF-orig', Pipeline([random_forest])))
pipelines.append(('ET-orig', Pipeline([extra_trees])))
pipelines.append(('Ada-orig', Pipeline([adaboost])))
pipelines.append(('GB-orig', Pipeline([gradient_boosting])))
pipelines.append(('Vot-orig', Pipeline([voting])))

# Dataset Padronizado
pipelines.append(('LR-padr', Pipeline([standard_scaler, reg_log])))
pipelines.append(('KNN-padr', Pipeline([standard_scaler, knn])))
pipelines.append(('CART-padr', Pipeline([standard_scaler, cart])))
pipelines.append(('NB-padr', Pipeline([standard_scaler, naive_bayes])))
pipelines.append(('SVM-padr', Pipeline([standard_scaler, svm])))
pipelines.append(('Bag-padr', Pipeline([standard_scaler, bagging])))
pipelines.append(('RF-padr', Pipeline([standard_scaler, random_forest])))
pipelines.append(('ET-padr', Pipeline([standard_scaler, extra_trees])))
pipelines.append(('Ada-padr', Pipeline([standard_scaler, adaboost])))
pipelines.append(('GB-padr', Pipeline([standard_scaler, gradient_boosting])))
pipelines.append(('Vot-padr', Pipeline([standard_scaler, voting])))

# Dataset Normalizado
pipelines.append(('LR-norm', Pipeline([min_max_scaler, reg_log])))
pipelines.append(('KNN-norm', Pipeline([min_max_scaler, knn])))
pipelines.append(('CART-norm', Pipeline([min_max_scaler, cart])))
pipelines.append(('NB-norm', Pipeline([min_max_scaler, naive_bayes])))
pipelines.append(('SVM-norm', Pipeline([min_max_scaler, svm])))
pipelines.append(('Bag-norm', Pipeline([min_max_scaler, bagging])))
pipelines.append(('RF-norm', Pipeline([min_max_scaler, random_forest])))
pipelines.append(('ET-norm', Pipeline([min_max_scaler, extra_trees])))
pipelines.append(('Ada-norm', Pipeline([min_max_scaler, adaboost])))
pipelines.append(('GB-norm', Pipeline([min_max_scaler, gradient_boosting])))
pipelines.append(('Vot-norm', Pipeline([min_max_scaler, voting])))

# Executando os pipelines
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std()) # formatando para 3 casas decimais
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(25,6))
fig.suptitle('Comparação dos Modelos - Dataset original, padronizado e normalizado')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names, rotation=90)
plt.show()

# Otimização dos hiperparâmetros

A otimização de hiperparâmetros é o processo de encontrar os valores ideais para os hiperparâmetros de um modelo de machine learning. O objetivo é encontrar a combinação de hiperparâmetros que resulta no melhor desempenho do modelo.


## Grid Search (*força bruta*)

Como Funciona o Grid Search?
1. Definição do Espaço de Hiperparâmetros: Primeiro, define-se um conjunto de valores possíveis para cada hiperparâmetro.
2. Avaliação das Combinações: Em seguida, cada combinação possível desses valores é avaliada.
3. Seleção do Melhor Conjunto: A combinação de hiperparâmetros que produz o melhor desempenho é selecionada.

In [ ]:
np.random.seed(7)  # Definindo uma semente global para este bloco

# Lista de modelos
models = []

# Criando os modelos e adicionando-os na lista de modelos
models.append(('LR', LogisticRegression(max_iter=200)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# Definindo os parâmetros do classificador base para o BaggingClassifier
base = DecisionTreeClassifier()
num_trees = 100
max_features = 3

# Criando os modelos para o VotingClassifier
bases = []
model1 = LogisticRegression(max_iter=200)
bases.append(('logistic', model1))
model2 = DecisionTreeClassifier()
bases.append(('cart', model2))
model3 = SVC()
bases.append(('svm', model3))

# Criando os ensembles e adicionando-os na lista de modelos
models.append(('Bagging', BaggingClassifier(estimator=base, n_estimators=num_trees)))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('Ada', AdaBoostClassifier(n_estimators=num_trees)))
models.append(('GB', GradientBoostingClassifier(n_estimators=num_trees)))
models.append(('Voting', VotingClassifier(estimators=bases, voting='hard')))

# Definindo os componentes do pipeline
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())

# Lista de pipelines
pipelines = []

# Criando pipelines para cada modelo
for name, model in models:
    pipelines.append((name + '-orig', Pipeline(steps=[(name, model)])))
    pipelines.append((name + '-padr', Pipeline(steps=[standard_scaler, (name, model)])))
    pipelines.append((name + '-norm', Pipeline(steps=[min_max_scaler, (name, model)])))

# Definindo os parâmetros para GridSearchCV
param_grids = {
    'LR': {
        'LR__C': [0.01, 0.1, 1, 10, 100],
        'LR__solver': ['liblinear', 'saga']
    },
    'KNN': {
        'KNN__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21],
        'KNN__metric': ["euclidean", "manhattan", "minkowski"]
    },
    'CART': {
        'CART__max_depth': [None, 10, 20, 30, 40, 50],
        'CART__min_samples_split': [2, 5, 10],
        'CART__min_samples_leaf': [1, 2, 4]
    },
    'NB': {
        'NB__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
    },
    'SVM': {
        #'SVM__C': [0.1, 1, 10, 100],
        'SVM__C': [1, 10],
        #'SVM__gamma': [1, 0.1, 0.01, 0.001],
        'SVM__gamma': [1, 0.1],
        'SVM__kernel': ['rbf', 'linear']
    },
    'RF': {
        'RF__n_estimators': [10, 50, 100, 200],
        'RF__max_features': ['auto', 'sqrt', 'log2'],
        'RF__max_depth': [None, 10, 20, 30],
        'RF__min_samples_split': [2, 5, 10],
        'RF__min_samples_leaf': [1, 2, 4]
    },
    'ET': {
        'ET__n_estimators': [10, 50, 100, 200],
        'ET__max_features': ['auto', 'sqrt', 'log2'],
        'ET__max_depth': [None, 10, 20, 30],
        'ET__min_samples_split': [2, 5, 10],
        'ET__min_samples_leaf': [1, 2, 4]
    },
    'Ada': {
        'Ada__n_estimators': [10, 50, 100, 200],
        'Ada__learning_rate': [0.01, 0.1, 1, 10]
    },
    'GB': {
        'GB__n_estimators': [10, 50, 100, 200],
        'GB__learning_rate': [0.01, 0.1, 0.2, 0.3],
        'GB__max_depth': [3, 5, 7, 9]
    },
    'Voting': {
        # Para VotingClassifier, geralmente não há hiperparâmetros para ajustar diretamente
        # Ajustar os hiperparâmetros dos estimadores base individualmente se necessário
    }
}

# Parâmetros de cross-validation e scoring
scoring = 'accuracy'
kfold = 5

# Executando o GridSearchCV para cada pipeline
for name, pipeline in pipelines:
    model_type = name.split('-')[0]
    if model_type in param_grids:
        param_grid = param_grids[model_type]
    else:
        param_grid = {}  # Para modelos que não têm parâmetros definidos

    grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid.fit(X_train, y_train)
    # Imprimindo a melhor configuração
    print("Modelo: %s - Melhor: %f usando %s" % (name, grid.best_score_, grid.best_params_))

In [ ]:
# Tuning do KNN

np.random.seed(7) # definindo uma semente global para este bloco

pipelines = []

# Definindo os componentes do pipeline
knn = ('KNN', KNeighborsClassifier())
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())

pipelines.append(('knn-orig', Pipeline(steps=[knn])))
pipelines.append(('knn-padr', Pipeline(steps=[standard_scaler, knn])))
pipelines.append(('knn-norm', Pipeline(steps=[min_max_scaler, knn])))

param_grid = {
    'KNN__n_neighbors': [1,3,5,7,9,11,13,15,17,19,21],
    'KNN__metric': ["euclidean", "manhattan", "minkowski"],
}

# Prepara e executa o GridSearchCV
for name, model in pipelines:
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid.fit(X_train, y_train)
    # imprime a melhor configuração
    print("Sem tratamento de missings: %s - Melhor: %f usando %s" % (name, grid.best_score_, grid.best_params_))

# Finalização do Modelo

In [ ]:
# Avaliação do modelo com o conjunto de testes
# Melhor modelo
# Modelo: GB-norm - Melhor: 0.823716 usando {'GB__learning_rate': 0.1, 'GB__max_depth': 3, 'GB__n_estimators': 50}

np.random.seed(7)

# Preparação do modelo
scaler = MinMaxScaler().fit(X_train)
rescaledX = scaler.transform(X_train) # aplicação da normalização no conjunto de treino
#model = RandomForestClassifier(n_estimators=100,
#                               max_features='sqrt',
#                               min_samples_split=5,
#                               max_depth=20,
#                               min_samples_leaf=1)
#model = GradientBoostingClassifier(n_estimators=50,
#                               learning_rate=0.1,
#                               max_depth=3)
#model = LogisticRegression(C=100,
#                           solver='liblinear')
model = KNeighborsClassifier(n_neighbors=21,
                           metric='manhattan')
model.fit(rescaledX, y_train)

# Estimativa da acurácia no conjunto de teste
rescaledTestX = scaler.transform(X_test) # aplicação da normalização no conjunto de teste
predictions = model.predict(rescaledTestX)
print(accuracy_score(y_test, predictions))

### Rodando o modelo a partir de um pipeline com os hiperparâmetros otimizados.

In [ ]:
np.random.seed(7)

#model = RandomForestClassifier(n_estimators=50,
#                               max_features='sqrt',
#                               min_samples_split=2,
#                               max_depth=10,
#                               min_samples_leaf=1)
#model = GradientBoostingClassifier(n_estimators=50,
#                               learning_rate=0.1,
#                               max_depth=3)
#model = LogisticRegression(C=100,
#                           solver='liblinear')
model = KNeighborsClassifier(n_neighbors=21,
                           metric='manhattan')

pipeline = Pipeline(steps=[('MinMaxScaler', MinMaxScaler()), ('RF', model)])

pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)
print(accuracy_score(y_test, predictions))

### Salvando os arquivos

In [ ]:
# Define o path
directory = 'models'

#Salvando os mapeamentos
if not os.path.exists(directory):
  os.makedirs(directory)

# Salvando o modelo
#model_filename = 'rf_flights_classifier.pkl'
#model_filename = 'gb_flights_classifier.pkl'
#model_filename = 'flights_lr.pkl'
model_filename = 'flights_knn.pkl'
with open(os.path.join(directory, model_filename), 'wb') as file:
    pickle.dump(model, file)

# Define o path
directory = 'scalers'

#Salvando os mapeamentos
if not os.path.exists(directory):
  os.makedirs(directory)

# Salvando o scaler
scaler_filename = 'minmax_scaler_flights.pkl'
with open(os.path.join(directory, scaler_filename), 'wb') as file:
    pickle.dump(scaler, file)

# Define o path
directory = 'pipelines'

#Salvando os mapeamentos
if not os.path.exists(directory):
  os.makedirs(directory)

# Salvando o pipeline
pipeline_filename = 'rf_flights_pipeline.pkl'
with open(os.path.join(directory, pipeline_filename), 'wb') as file:
    pickle.dump(pipeline, file)

# Salvando X_test e y_test
column_names = ['DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DEPARTURE_DELAY', 'SCHEDULED_ARRIVAL']
X_test_df = pd.DataFrame(X_test, columns=column_names)
column_names = ['DELAY_DETECTED']
y_test_df = pd.DataFrame(y_test, columns=column_names)
test_dataset_flights_df = pd.concat([X_test_df, y_test_df], axis=1)

# Define o path
directory = 'data'

#Salvando os mapeamentos
if not os.path.exists(directory):
  os.makedirs(directory)

X_test_df.to_csv(os.path.join(directory, "X_test_dataset_flights.csv"), index=False)
y_test_df.to_csv(os.path.join(directory, "y_test_dataset_flights.csv"), index=False)
test_dataset_flights_df.to_csv(os.path.join(directory, "test_dataset_flights.csv"), index=False)

## Simulando a aplicação do modelo em dados não vistos

In [ ]:
# Preparação do modelo com TODO o dataset
scaler = MinMaxScaler().fit(X) # ajuste do scaler com TODO o dataset
rescaledX = scaler.transform(X) # aplicação da normalização com TODO o dataset
model.fit(rescaledX, y)

In [ ]:
# Novos dados - não sabemos a classe!
data = {'day':  [1, 12, 18],
        'week': [1, 4, 6],
        'airline': [3, 5, 10],
        'flight_no': [100, 500, 800],
        'tail': [500, 1500, 2000],
        'origin': [50, 100, 200],
        'destination': [80, 120, 210],
        'dep_delay': [5, -8, 12],
        'schedule_arrival': [415, 1309, 1845],
        }

atributos = ['day', 'week', 'airline', 'flight_no', 'tail', 'origin', 'destination', 'dep_delay', 'schedule_arrival']
entrada = pd.DataFrame(data, columns=atributos)

array_entrada = entrada.values
X_entrada = array_entrada[:,0:9].astype(float)

# Padronização nos dados de entrada usando o scaler utilizado em X
rescaledEntradaX = scaler.transform(X_entrada)
print(rescaledEntradaX)

In [ ]:
# Predição de classes dos dados de entrada
saidas = model.predict(rescaledEntradaX)
print(saidas)

#Max e Min

In [ ]:
print("Max")
print(test_dataset_flights_df.max())
print("Min")
print(test_dataset_flights_df.min())